In [1]:
__imp

Libraries have been loaded


## Load Additional Libraries

In [9]:
from sklearn.model_selection import train_test_split
basepath = os.path.expanduser('../')

In [8]:
train = pd.read_csv('../data/raw/application_train.csv.zip')

### Prepare cv indices 

In [13]:
folds        = ['F0', 'F1', 'F2', 'F3', 'F4', 'F5']
fold_indices = {}

for index, fold in enumerate(folds):
    itr, _ = train_test_split(range(len(train)), test_size=.2, random_state=SEED+index)
    fold_indices[f'F{index}'] = itr

In [15]:
pd.DataFrame(fold_indices).to_csv('../data/raw/cv_idx.csv', index=False)

In [2]:
ytrue = np.load('../data/interim/dataset1/true.npy')
ypred = np.load('../data/interim/dataset1/preds.npy')

assert len(ytrue) == len(ypred)

In [3]:
from sklearn.metrics import roc_auc_score

In [4]:
roc_auc_score(ytrue, ypred)

0.4572496897807594

In [5]:
ypred

array([0.18175554, 0.09738664, 0.1439396 , ..., 0.17422187, 0.13351094,
       0.14616917])